In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import requests
import calendar
import time
from datetime import date, timedelta

class PlusPortals:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.driver = self._initialize_driver()

    def _initialize_driver(self):
        try:
            driver = webdriver.Chrome()
        except:
            ChromeDriverManager().install()
            driver = webdriver.Chrome()
        return driver

    def login(self):
        self.driver.get('https://plusportals.com/springcreekacademy')
        self.driver.find_element(By.ID, "UserName").send_keys(self.username)
        self.driver.find_element(By.ID, "Password").send_keys(self.password)
        submit_btn = self.driver.find_element(By.CSS_SELECTOR, 'input[type="submit"][value="Sign in"]')
        submit_btn.click()

    def set_pending_coursework(self):
        coursework_menu = self.driver.find_element(By.CSS_SELECTOR, "span.k-widget.k-dropdown.k-header[aria-owns='mCourseworkRangeId_1728_listbox']")
        coursework_menu.click()
        pending_coursework = self.driver.find_element(By.XPATH, "//li[text()='Pending Coursework']")
        pending_coursework.click()

    def find_xpath(self, x):
        xpath = '//*[@id="gridCoursework_1728"]/div[2]/table/tbody/tr[1]/td[' + str(x) + ']'
        result = self.driver.find_element(By.XPATH, xpath).text
        return result

    def any_tasks(self):
        try:
            self.find_xpath(1)
            return True
        except:
            return False

    def mark_complete(self):
        threedots = '//*[@id="gridCoursework_1728"]/div[2]/table/tbody/tr[1]/td[7]/img'
        self.wait.until(EC.element_to_be_clickable((By.XPATH, threedots))).click()
        completed_txt = '//*[@id="context-menu"]/li[3]/span'
        self.wait.until(EC.element_to_be_clickable((By.XPATH, completed_txt))).click()

    def process_row(self):
        a_course = self.find_xpath(2)
        a_title = self.find_xpath(3)
        a_details = self.find_xpath(4)
        a_due = self.find_xpath(6)

        task = {
            'course': a_course,
            'title': a_title,
            'details': a_details,
            'due': a_due
        }

        return task

    def posix_date(self, due):
        def click_time(entry):
            result = calendar.timegm(time.strptime(entry + ' 12:15:00', '%m-%d-%Y %H:%M:%S')) * 1000
            return result

        if due == 'Today':
            due = date.today().strftime('%m-%d-%Y')
        elif due == 'Tomorrow':
            due = (date.today() + timedelta(days=1)).strftime('%m-%d-%Y')

        start_date = click_time(date.today().strftime('%m-%d-%Y'))
        due_date = click_time(due)

        result = {"start": start_date, "due": due_date}
        return result

    def create_clickup_task(self, task, clickup):
        c2tag = {
            '9PERFIN 5759/02': 'personal finance',
            '11ENG 3150/01': 'english',
            '11ALG2H 3351/01': 'algebra 2',
            '11USHIST 3450/01': 'us history',
            '11SPN3H 3551/01': 'spanish'
        }
        dates = self.posix_date(task['due'])

        task_data = {
            "name": task['title'],
            "description": task['details'],
            "status": "BACKLOG",
            "priority": 3,
            "due_date": dates['due'],
            "tags": [c2tag[task['course']]]
        }

        response = clickup.create_task(task_data)

        if response.status_code == 200:
            print("Task created successfully!")
            task_info = response.json()
            print("Task ID:", task_info["id"])
            return True
        else:
            print("Error creating task. Status code:", response.status_code)
            print("Response:", response.text)
            return False

    def run(self, clickup):
        self.login()
        self.set_pending_coursework()

        while self.any_tasks():
            task = self.process_row()
            if self.create_clickup_task(task, clickup):
                self.mark_complete()
                worked = self.ok(3)
                if not worked:
                    input("Click OK then press Enter to continue.")
            else:
                break
                print("Error creating ClickUp task")

        self.driver.quit()

    def ok(self, method=1):
        try:
            if method == 1:
                self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btnOK'))).click()
            # Add other ok methods here

            elif method==2:
                from selenium.webdriver.common.action_chains import ActionChains
                button = driver.find_element(By.XPATH, "//button[text()='OK']")
                ActionChains(driver).move_to_element(button).click().perform()
        
            elif method==3:
                button = driver.find_element(By.CLASS_NAME, 'btnOK')

                # Scroll to the OK button
                driver.execute_script("arguments[0].scrollIntoView();", button)
                button.click()

            elif method==4:
                button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')
                ## Scroll to the OK button
                #driver.execute_script("arguments[0].scrollIntoView();", button)
                button.click()

            elif method==5:
                button = driver.find_element(By.CLASS_NAME, 'btnOK k-button ')

                relative = '//*[@id="top"]/div[29]/div[2]/button'
                absolute = '/html/body/div[29]/div[2]/button'
                button = driver.find_element(By.XPATH, absolute)

                ## Scroll to the OK button
                #driver.execute_script("arguments[0].scrollIntoView();", button)
                button.click()

        except:
            print("OK button was not clicked")
            return False

class ClickUp:
    def __init__(self, api_token, space_id, list_id):
        self.api_token = api_token
        self.space_id = space_id
        self.list_id = list_id
        self.base_url = "https://api.clickup.com/api/v2"
        self.tasks_url = f"{self.base_url}/list/{self.list_id}/task"
        self.headers = {
            "Authorization": api_token,
            "Content-Type": "application/json"
        }

    def create_task(self, task_data):
        response = requests.post(self.tasks_url, headers=self.headers, json=task_data)
        return response

In [2]:
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

username = config.get("username")
password = config.get("password")
api_token = config.get("api")  # ClickUp API token

In [3]:
plusportals = PlusPortals(username, password)

In [4]:
clickup = ClickUp(api_token, '9006014177', '901100521675')

In [5]:
plusportals.run(clickup)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=116.0.5845.140)
Stacktrace:
0   chromedriver                        0x000000010116e65c chromedriver + 4318812
1   chromedriver                        0x0000000101166d00 chromedriver + 4287744
2   chromedriver                        0x0000000100d98644 chromedriver + 296516
3   chromedriver                        0x0000000100dd76ac chromedriver + 554668
4   chromedriver                        0x0000000100dcbf90 chromedriver + 507792
5   chromedriver                        0x0000000100dcb858 chromedriver + 505944
6   chromedriver                        0x0000000100e0ed28 chromedriver + 781608
7   chromedriver                        0x0000000100dca178 chromedriver + 500088
8   chromedriver                        0x0000000100dcafc0 chromedriver + 503744
9   chromedriver                        0x000000010112ec40 chromedriver + 4058176
10  chromedriver                        0x0000000101133160 chromedri

In [ ]:
def main():
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)

    username = config.get("username")
    password = config.get("password")
    api_token = config.get("api")  # ClickUp API token

    plusportals = PlusPortals(username, password)
    clickup = ClickUp(api_token, '9006014177', '901100521675')
    plusportals.run(clickup)

if __name__ == "__main__":
    main()

### Tasks to finish

1. Figure out how to add Ben's user id number